In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("data/dataset.csv")
adjacent_counties = pd.read_csv('data/adjacent_counties.csv')

In [ ]:
adjacent_counties

In [ ]:
# get modes of categories
def get_mode(inp):
    return list(inp.value_counts().index)[0]

In [ ]:
data.drop(columns = ['loc', 'h3']).groupby('fips').agg(get_mode)

In [2]:
def bin_homogeneity(dataset, to_groupby = ['fips'], bins = ['lc_type', 'bldgtype']):
    totals = dataset.drop(columns = ['loc', 'h3']).groupby(to_groupby).count().rename(columns = {'st_damcat' : 'total'}).iloc[:, 0]
    relative_freq = dataset.drop(columns = ['loc', 'h3']).groupby(to_groupby + bins).count().reset_index().merge(totals, how = 'left', on = to_groupby)
    #print(relative_freq.columns)
    to_drop = [c for c in relative_freq.columns if c not in to_groupby + ['freq'] + bins]
    relative_freq = relative_freq.assign(freq = relative_freq.st_damcat / relative_freq.total).drop(columns = to_drop).sort_values(by = to_groupby + ['freq'], ascending = False)
    def filter_cdf(col):
        #print(np.array(col.freq))
        _max = sum(np.cumsum(np.array(col.freq)) <= 0.8)
        return col[:_max]
    most_freq = relative_freq.groupby(to_groupby).apply(include_groups = False, func = filter_cdf).reset_index()
    most_freq = most_freq.drop(columns = [c for c in most_freq.columns if 'level' in c])
    #print(most_freq)
    return most_freq
    
bin_categories = ['lc_type', 'bldgtype']
most_freq = bin_homogeneity(data, bin_categories)
most_freq.to_csv()

NameError: name 'data' is not defined

In [ ]:
# using homogeneity counts for Bigquery analysis
most_freq.groupby(['fips']).count().reset_index().iloc[:, :2].rename(columns = {'lc_type' : 'count'})#.to_csv('eda_lc_type_bldgtype_homogeneity.csv')